In [1]:
!kaggle datasets download -d puneet6060/intel-image-classification

!unzip intel-image-classification.zip
!rm intel-image-classification.zip
!pip install gradio


Streaming output truncated to the last 5000 lines.
  inflating: seg_train/seg_train/mountain/8484.jpg  
  inflating: seg_train/seg_train/mountain/8485.jpg  
  inflating: seg_train/seg_train/mountain/8486.jpg  
  inflating: seg_train/seg_train/mountain/8488.jpg  
  inflating: seg_train/seg_train/mountain/8498.jpg  
  inflating: seg_train/seg_train/mountain/85.jpg  
  inflating: seg_train/seg_train/mountain/8500.jpg  
  inflating: seg_train/seg_train/mountain/8532.jpg  
  inflating: seg_train/seg_train/mountain/8536.jpg  
  inflating: seg_train/seg_train/mountain/8549.jpg  
  inflating: seg_train/seg_train/mountain/8561.jpg  
  inflating: seg_train/seg_train/mountain/8564.jpg  
  inflating: seg_train/seg_train/mountain/857.jpg  
  inflating: seg_train/seg_train/mountain/8571.jpg  
  inflating: seg_train/seg_train/mountain/8573.jpg  
  inflating: seg_train/seg_train/mountain/8587.jpg  
  inflating: seg_train/seg_train/mountain/8594.jpg  
  inflating: seg_train/seg_train/mountain/860.jpg  

In [8]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.5 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, GlobalAveragePooling2D, Input, BatchNormalization, Add
from tensorflow.keras.applications import VGG16, InceptionV3, MobileNetV2, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall

In [9]:

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch


train_dir = '/content/seg_train/seg_train'
test_dir = '/content/seg_test/seg_test'

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE = 150, 150, 32

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)



Found 14034 images belonging to 6 classes.


<ipython-input-9-d952d9807492>:13: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


Found 3000 images belonging to 6 classes.


In [4]:
from tensorflow.keras.metrics import Precision, Recall


In [ ]:
#  CNN Model
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), padding='same', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
        Activation('relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),

        Conv2D(64, (3, 3), padding='same'),
        Activation('relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),

        Conv2D(128, (3, 3), padding='same'),
        Activation('relu'),
        MaxPooling2D(2, 2),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(6, activation='softmax')
    ])
    return model

cnn_model = create_model()
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

In [ ]:
checkpoint_path = "cnn_model_best.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:
history = cnn_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[ReduceLROnPlateau(), checkpoint]
)

Epoch 1/10
439/439 [==============================] - ETA: 0s - loss: 1.2656 - accuracy: 0.5063 - precision: 0.6971 - recall: 0.3026
Epoch 1: val_accuracy improved from -inf to 0.66300, saving model to cnn_model_best.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


439/439 [==============================] - 113s 242ms/step - loss: 1.2656 - accuracy: 0.5063 - precision: 0.6971 - recall: 0.3026 - val_loss: 0.8789 - val_accuracy: 0.6630 - val_precision: 0.7814 - val_recall: 0.4860 - lr: 0.0010
Epoch 2/10
439/439 [==============================] - ETA: 0s - loss: 0.9475 - accuracy: 0.6335 - precision: 0.7340 - recall: 0.4958
Epoch 2: val_accuracy improved from 0.66300 to 0.70733, saving model to cnn_model_best.h5
439/439 [==============================] - 105s 239ms/step - loss: 0.9475 - accuracy: 0.6335 - precision: 0.7340 - recall: 0.4958 - val_loss: 0.7949 - val_accuracy: 0.7073 - val_precision: 0.8331 - val_recall: 0.5357 - lr: 0.0010
Epoch 3/10
439/439 [==============================] - ETA: 0s - loss: 0.8142 - accuracy: 0.6890 - precision: 0.7601 - recall: 0.5875
Epoch 3: val_accuracy improved from 0.70733 to 0.75833, saving model to cnn_model_best.h5
439/439 [==============================] - 111s 252ms/step - loss: 0.8142 - accuracy: 0.6890 -

In [ ]:
best_cnn_model.evaluate(validation_generator)

94/94 [==============================] - 4s 39ms/step - loss: 0.4785 - accuracy: 0.8360 - precision: 0.8572 - recall: 0.8123


[0.4785269796848297, 0.8360000252723694, 0.857193112373352, 0.812333345413208]

In [ ]:
loaded_model = tf.keras.models.load_model('/content/cnn_model_best.h5')
results = loaded_model.evaluate(validation_generator)
print("Test Results - Loss: {:.4f}, Accuracy: {:.2f}%".format(results[0], results[1]*100))

94/94 [==============================] - 5s 41ms/step - loss: 0.4785 - accuracy: 0.8360 - precision: 0.8572 - recall: 0.8123
Test Results - Loss: 0.4785, Accuracy: 83.60%


In [ ]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
for layer in base_model.layers:
    layer.trainable = False

vgg_model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'), Dropout(0.5),
    Dense(6, activation='softmax')
])

vgg_model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall()])

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg_checkpoint_path = "/content/vgg16_model_best.h5"
vgg_checkpoint = ModelCheckpoint(vgg_checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)


In [ ]:
vgg_history = vgg_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[vgg_checkpoint]
)

Epoch 1/10
439/439 [==============================] - ETA: 0s - loss: 0.7399 - accuracy: 0.7194 - precision: 0.8066 - recall: 0.6192
Epoch 1: val_accuracy improved from -inf to 0.85200, saving model to /content/vgg16_model_best.h5
439/439 [==============================] - 114s 254ms/step - loss: 0.7399 - accuracy: 0.7194 - precision: 0.8066 - recall: 0.6192 - val_loss: 0.4059 - val_accuracy: 0.8520 - val_precision: 0.8777 - val_recall: 0.8230
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


439/439 [==============================] - ETA: 0s - loss: 0.5141 - accuracy: 0.8130 - precision: 0.8500 - recall: 0.7723
Epoch 2: val_accuracy improved from 0.85200 to 0.86633, saving model to /content/vgg16_model_best.h5
439/439 [==============================] - 108s 246ms/step - loss: 0.5141 - accuracy: 0.8130 - precision: 0.8500 - recall: 0.7723 - val_loss: 0.3687 - val_accuracy: 0.8663 - val_precision: 0.8924 - val_recall: 0.8433
Epoch 3/10
439/439 [==============================] - ETA: 0s - loss: 0.4715 - accuracy: 0.8292 - precision: 0.8576 - recall: 0.7942
Epoch 3: val_accuracy did not improve from 0.86633
439/439 [==============================] - 109s 248ms/step - loss: 0.4715 - accuracy: 0.8292 - precision: 0.8576 - recall: 0.7942 - val_loss: 0.3510 - val_accuracy: 0.8653 - val_precision: 0.8879 - val_recall: 0.8477
Epoch 4/10
439/439 [==============================] - ETA: 0s - loss: 0.4430 - accuracy: 0.8393 - precision: 0.8676 - recall: 0.8061
Epoch 4: val_accuracy impr

In [ ]:
best_vgg_model = tf.keras.models.load_model(vgg_checkpoint_path)
test_results = best_vgg_model.evaluate(validation_generator)
print(f"VGG16 Model - Test Loss: {test_results[0]}, Test Accuracy: {test_results[1]*100:.2f}%, Precision: {test_results[2]*100:.2f}%, Recall: {test_results[3]*100:.2f}%")

94/94 [==============================] - 7s 67ms/step - loss: 0.3066 - accuracy: 0.8817 - precision: 0.8974 - recall: 0.8713
VGG16 Model - Test Loss: 0.3066413104534149, Test Accuracy: 88.17%, Precision: 89.74%, Recall: 87.13%


In [ ]:
best_vgg_model.save('/content/vgg16_model_best_final.h5')

loaded_vgg_model = tf.keras.models.load_model('/content/vgg16_model_best_final.h5')
final_results = loaded_vgg_model.evaluate(validation_generator)
print(f"Final VGG16 Model - Loss: {final_results[0]}, Accuracy: {final_results[1]*100:.2f}%, Precision: {final_results[2]*100:.2f}%, Recall: {final_results[3]*100:.2f}%")

94/94 [==============================] - 7s 67ms/step - loss: 0.3066 - accuracy: 0.8817 - precision: 0.8974 - recall: 0.8713
Final VGG16 Model - Loss: 0.3066413104534149, Accuracy: 88.17%, Precision: 89.74%, Recall: 87.13%


In [ ]:
resnet50_model = Sequential([
    ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(150, 150, 3))),
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
resnet50_checkpoint = ModelCheckpoint('/content/resnet50_model_best.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
resnet50_history = resnet50_model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[resnet50_checkpoint])


94765736/94765736 [==============================] - 6s 0us/step
Epoch 1/10
439/439 [==============================] - ETA: 0s - loss: 0.7827 - accuracy: 0.7530 - precision_1: 0.8078 - recall_1: 0.6803
Epoch 1: val_accuracy improved from -inf to 0.18367, saving model to /content/resnet50_model_best.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


439/439 [==============================] - 162s 286ms/step - loss: 0.7827 - accuracy: 0.7530 - precision_1: 0.8078 - recall_1: 0.6803 - val_loss: 1.9227 - val_accuracy: 0.1837 - val_precision_1: 0.2857 - val_recall_1: 6.6667e-04
Epoch 2/10
439/439 [==============================] - ETA: 0s - loss: 0.5208 - accuracy: 0.8286 - precision_1: 0.8613 - recall_1: 0.7927
Epoch 2: val_accuracy improved from 0.18367 to 0.69600, saving model to /content/resnet50_model_best.h5
439/439 [==============================] - 126s 287ms/step - loss: 0.5208 - accuracy: 0.8286 - precision_1: 0.8613 - recall_1: 0.7927 - val_loss: 0.9990 - val_accuracy: 0.6960 - val_precision_1: 0.7275 - val_recall_1: 0.6453
Epoch 3/10
439/439 [==============================] - ETA: 0s - loss: 0.4447 - accuracy: 0.8509 - precision_1: 0.8763 - recall_1: 0.8251
Epoch 3: val_accuracy improved from 0.69600 to 0.71667, saving model to /content/resnet50_model_best.h5
439/439 [==============================] - 126s 286ms/step - los

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
resnet50_model.save('/content/resnet50_model_final.h5')


In [ ]:
final_resnet50_model = tf.keras.models.load_model('/content/resnet50_model_final.h5')
resnet50_results = final_resnet50_model.evaluate(validation_generator)
print(f"ResNet50 Model - Loss: {resnet50_results[0]}, Accuracy: {resnet50_results[1]*100:.2f}%, Precision: {resnet50_results[2]*100:.2f}%, Recall: {resnet50_results[3]*100:.2f}%")

94/94 [==============================] - 7s 66ms/step - loss: 0.7467 - accuracy: 0.7620 - precision_1: 0.7806 - recall_1: 0.7460
ResNet50 Model - Loss: 0.7467337250709534, Accuracy: 76.20%, Precision: 78.06%, Recall: 74.60%


In [18]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint

base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
base_model.trainable = False

inception_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

inception_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

inception_checkpoint = ModelCheckpoint('/content/inception_model_best.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

inception_history = inception_model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[inception_checkpoint])

for layer in base_model.layers[-50:]:
    layer.trainable = True

inception_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

inception_history_fine_tune = inception_model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[inception_checkpoint])

87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/10
439/439 [==============================] - ETA: 0s - loss: 0.6176 - accuracy: 0.7872 - precision_1: 0.8224 - recall_1: 0.7352
Epoch 1: val_accuracy improved from -inf to 0.88067, saving model to /content/inception_model_best.h5
439/439 [==============================] - 126s 231ms/step - loss: 0.6176 - accuracy: 0.7872 - precision_1: 0.8224 - recall_1: 0.7352 - val_loss: 0.3255 - val_accuracy: 0.8807 - val_precision_1: 0.8952 - val_recall_1: 0.8573
Epoch 2/10
439/439 [==============================] - ETA: 0s - loss: 0.4616 - accuracy: 0.8347 - precision_1: 0.8573 - recall_1: 0.8043
Epoch 2: val_accuracy improved from 0.88067 to 0.88533, saving model to /content/inception_model_best.h5
439/439 [==============================] - 96s 220ms/step - loss: 0.4616 - accuracy: 0.8347 - precision_1: 0.8573 - recall_1: 0.8043 - val_loss: 0.3043 - val_accuracy: 0.8853 - val_precision_1: 0.8982 - val_recall_1: 0.8703
Epoch

In [19]:
inception_model.save('/content/inception_model_fine_tuned.h5')


In [ ]:
inception_model.save('/content/inception_model_final.h5')

In [23]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

mobilenet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

mobilenet_model.compile(optimizer=Adam(learning_rate=0.001),
                        loss='categorical_crossentropy',
                        metrics=['accuracy', Precision(), Recall()])

for layer in base_model.layers[-50:]:
    layer.trainable = True

mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001),
                        loss='categorical_crossentropy',
                        metrics=['accuracy', Precision(), Recall()])

mobilenet_checkpoint = ModelCheckpoint('/content/mobilenet_model_best.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=0.00001)

mobilenet_history = mobilenet_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[mobilenet_checkpoint, reduce_lr]
)

mobilenet_model.save('/content/mobilenet_model_final.h5')


9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/10
439/439 [==============================] - ETA: 0s - loss: 1.0394 - accuracy: 0.6103 - precision_4: 0.7273 - recall_4: 0.4699
Epoch 1: val_accuracy improved from -inf to 0.82100, saving model to /content/mobilenet_model_best.h5
439/439 [==============================] - 99s 213ms/step - loss: 1.0394 - accuracy: 0.6103 - precision_4: 0.7273 - recall_4: 0.4699 - val_loss: 0.4661 - val_accuracy: 0.8210 - val_precision_4: 0.8422 - val_recall_4: 0.7987 - lr: 1.0000e-04
Epoch 2/10
439/439 [==============================] - ETA: 0s - loss: 0.6486 - accuracy: 0.7604 - precision_4: 0.8107 - recall_4: 0.6987
Epoch 2: val_accuracy improved from 0.82100 to 0.85233, saving model to /content/mobilenet_model_best.h5
439/439 [==============================] - 91s 207ms/step - loss: 0.6486 - accuracy: 0.7604 - precision_4: 0.8107 - recall_4: 0.6987 - val_loss: 0.3977 - val_accuracy: 0.8523 - val_precision_4: 0.8661 - val_recall_4

In [ ]:
mobilenet_model.save('/content/mobilenet_model_final1.h5')

In [24]:
import gdown

url_template = "https://drive.google.com/uc?id={}"

file_ids = [
    "1Xmy7BkiN4r_WQg3CycXNVXvve62RboAy",
    "1QN8kPQk7-oFYaYoPPBxccqCHqQUNjJRp",
    "1Sair8rS8wcMRpAOkgDz-RlxmT2DaicuK",
    "1p5nOrJEDuZq6RNHb2ccXaVOUcpCLYy80",
    "1UPnLHOItK14474dVoALDYRb4VshwcnWy"
]

output_files = [
    "/content/cnn_model_final.h5",
    "/content/vgg16_model_final.h5",
    "/content/resnet50_model_final.h5",
    "/content/inceptionv3_model_final.h5",
    "/content/mobilenet_model_final.h5"
]

for file_id, output_file in zip(file_ids, output_files):
    url = url_template.format(file_id)
    gdown.download(url, output_file, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1Xmy7BkiN4r_WQg3CycXNVXvve62RboAy
From (redirected): https://drive.google.com/uc?id=1Xmy7BkiN4r_WQg3CycXNVXvve62RboAy&confirm=t&uuid=d48e33eb-a22f-42f1-9954-5af4ece6e105
To: /content/cnn_model_final.h5
100%|██████████| 256M/256M [00:01<00:00, 145MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QN8kPQk7-oFYaYoPPBxccqCHqQUNjJRp
To: /content/vgg16_model_final.h5
100%|██████████| 84.1M/84.1M [00:01<00:00, 72.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Sair8rS8wcMRpAOkgDz-RlxmT2DaicuK
From (redirected): https://drive.google.com/uc?id=1Sair8rS8wcMRpAOkgDz-RlxmT2DaicuK&confirm=t&uuid=0ccab7a7-afa8-4b98-937f-78af193387da
To: /content/resnet50_model_final.h5
100%|██████████| 290M/290M [00:04<00:00, 70.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1p5nOrJEDuZq6RNHb2ccXaVOUcpCLYy80
To: /content/inceptionv3_model_final.h5
100%|██████████| 94.1M/94.1M [00:01<00:00, 92.0MB/s]
Downloading.

In [29]:
import tensorflow as tf
from sklearn.metrics import classification_report

models = {
    "CNN": tf.keras.models.load_model('/content/cnn_model_final.h5'),
    "VGG16": tf.keras.models.load_model('/content/vgg16_model_final.h5'),
    "ResNet50": tf.keras.models.load_model('/content/resnet50_model_final.h5'),
    "InceptionV3": tf.keras.models.load_model('/content/inceptionv3_model_final.h5'),
    "MobileNetV2": tf.keras.models.load_model('/content/mobilenet_model_final.h5')
}

results = {}
for model_name, model in models.items():
    loss, accuracy, precision, recall = model.evaluate(validation_generator, verbose=0)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    results[model_name] = {
        "Loss": loss,
        "Accuracy": accuracy * 100,
        "Precision": precision * 100,
        "Recall": recall * 100,
        "F1 Score": f1_score * 100
    }

print("Evaluation Metrics:\n")
print("{:<15} {:<10} {:<10} {:<10} {:<10} {:<10}".format("Model", "Loss", "Accuracy", "Precision", "Recall", "F1 Score"))
for model_name, metrics in results.items():
    print("{:<15} {:.4f}     {:.2f}%     {:.2f}%      {:.2f}%      {:.2f}%".format(
        model_name,
        metrics["Loss"],
        metrics["Accuracy"],
        metrics["Precision"],
        metrics["Recall"],
        metrics["F1 Score"]
    ))


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 528, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1908, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1485, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

1/1 [==============================] - 5s 5s/step
Evaluation Metrics:

Model           Loss       Accuracy   Precision  Recall     F1 Score  
CNN             0.4785     83.60%     85.72%      81.23%      83.42%
VGG16           0.3066     88.17%     89.74%      87.13%      88.42%
ResNet50        0.5779     82.67%     84.89%      81.43%      83.12%
InceptionV3     0.2966     89.27%     90.46%      87.53%      88.97%
MobileNetV2     0.3518     87.07%     88.56%      85.17%      86.83%


In [ ]:
final_vgg16_model = tf.keras.models.load_model('/content/vgg16_model_best_final.h5')
vgg16_results = final_vgg16_model.evaluate(validation_generator)
print(f"VGG16 Model - Loss: {vgg16_results[0]}, Accuracy: {vgg16_results[1]*100:.2f}%, Precision: {vgg16_results[2]*100:.2f}%, Recall: {vgg16_results[3]*100:.2f}%")

94/94 [==============================] - 7s 66ms/step - loss: 0.3066 - accuracy: 0.8817 - precision: 0.8974 - recall: 0.8713
VGG16 Model - Loss: 0.3066413104534149, Accuracy: 88.17%, Precision: 89.74%, Recall: 87.13%


In [ ]:
import tensorflow as tf

def load_model(model_path):
    try:
        model = tf.keras.models.load_model(model_path)
        print("Model loaded successfully from", model_path)
        return model
    except Exception as e:
        print("Failed to load model:", e)

model_path = '/content/cnn_model_best.h5'
model = load_model(model_path)
if model:
    print("Model is ready for predictions.")


Model loaded successfully from /content/cnn_model_best.h5
Model is ready for predictions.


In [32]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

def load_model(model_path):
    """Load a TensorFlow model from a given path."""
    return tf.keras.models.load_model(model_path)

models = {
    "CustomCNN": load_model("/content/cnn_model_final.h5"),
    "VGG16": load_model("/content/vgg16_model_final.h5"),
    "ResNet50": load_model("/content/resnet50_model_final.h5"),
    "InceptionV3": load_model("/content/inceptionv3_model_final.h5"),
    "MobileNetV2": load_model("/content/mobilenet_model_final.h5")
}

def classify_image(model, img, resize_shape):
    """Classify an image using the given model and resize dimensions."""
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    img = img.resize(resize_shape)
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
    return {class_names[i]: float(prediction[0][i]) for i in range(6)}

def get_prediction(model_name, img):
    """Handle model selection and image resizing before classification."""
    if model_name in ["MobileNetV2"]:
        resize_shape = (224, 224)
    else:
        resize_shape = (150, 150)
    model = models[model_name]
    return classify_image(model, img, resize_shape)

css = """
body { background-color: white; }
h1, h2, h3, h4 { color: white; }
.tab-content { background-color: white; border: 1px solid #ccc; }
.label, .button, .dropdown { width: 100%; }
"""

def setup_interface():
    with gr.Blocks(css=css) as app:
        with gr.Tab("Home"):
            gr.Markdown("# Image Classification Project")
            gr.Markdown("Explore the application to classify images into categories such as buildings, forests, glaciers, mountains, seas, and streets.")
            gr.Markdown("## Evaluation Metrics:")
            gr.Markdown("""
                | Model        | Loss   | Accuracy | Precision | Recall | F1 Score |
                |--------------|--------|----------|-----------|--------|----------|
                | CNN          | 0.4785 | 83.60%   | 85.72%    | 81.23% | 83.42%   |
                | VGG16        | 0.3066 | 88.17%   | 89.74%    | 87.13% | 88.42%   |
                | ResNet50     | 0.5779 | 82.67%   | 84.89%    | 81.43% | 83.12%   |
                | InceptionV3  | 0.2966 | 89.27%   | 90.46%    | 87.53% | 88.97%   |
                | MobileNetV2  | 0.3518 | 87.07%   | 88.56%    | 85.17% | 86.83%   |
            """)
        with gr.Tab("Classify"):
            with gr.Column(scale=1):
                img_input = gr.Image(label="Upload Image")
                model_choice = gr.Dropdown(list(models.keys()), label="Choose Model", value="CustomCNN")
                submit_button = gr.Button("Classify")
                label_output = gr.Label()
                submit_button.click(fn=get_prediction, inputs=[model_choice, img_input], outputs=label_output)

    return app

app = setup_interface()
app.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://69ef0f08f794c00521.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


SyntaxError: invalid syntax (<ipython-input-20-1ccd82e57bfd>, line 1)